<a href="https://colab.research.google.com/github/robert-sturrock/publishing-paid-me/blob/master/Publishing_paid_me_Goodreads_Data_(%2BGitHub_integration).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Publishing paid me - Goodreads Data

Aim of this workbook is to get a list of all Goodreads data that will roughly match to the Publishing Paid Me dataset. 

In [ ]:
#@title Set up code here{display-mode: "form"} 
import numpy as np
import matplotlib as m
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import re as re
import pandas as pd
import unicodedata 
import requests
import seaborn as sns
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Loading package for % on axis
import matplotlib.ticker as mtick
from matplotlib.dates import DateFormatter

# set style
# NOTE: without this the other formatting options (eg. dpi) don't actually work
# Load packages for plotting / styling
import matplotlib.style as style
# print(plt.style.available)
import seaborn as sns
style.use('default')

In [ ]:
# Limiting floats output to 3 decimal points
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x)) 

## Sync with GitHub

Mount drive and Github

In [ ]:
from google.colab import drive
from os.path import join

ROOT = '/content/drive'     # default for the drive
PROJ = 'My Drive/Data Projects/Publishing Paid Me'

GIT_USERNAME = "robert-sturrock" 
GIT_TOKEN = "50391de799dba17eec450691a028bc5319e4412a" 
GIT_REPOSITORY = "publishing-paid-me"    


drive.mount(ROOT)  # we mount the drive at /content/drive

PROJECT_PATH = join(ROOT, PROJ)

GIT_PATH = f"https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git"
!mkdir ./tmp/publishing-paid-me
!git clone "{GIT_PATH}" ./tmp/publishing-paid-me
!mv ./tmp/publishing-paid-me/* "{PROJECT_PATH}"
!rm -r ./tmp/publishing-paid-me

# not sure what rsync is doing here. But I think it's copying the files back to the root
# !rsync -a -P --exclude=data/ "{PROJECT_PATH}"/* ./

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Cloning into './tmp/publishing-paid-me'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 40 (delta 11), reused 14 (delta 3), pack-reused 0
Unpacking objects: 100% (40/40), done.


Check files in drive now

In [ ]:
!ls "{PROJECT_PATH}"

books.csv  LICENSE  ppm_20201010.csv


## Create Goodreads Data


Data stored in Drive / Github

In [ ]:
file_path = f"{PROJECT_PATH}/ppm_20201010.csv"
df = pd.read_csv(file_path)

Rename all of the columns

In [ ]:
cols = ['timestamp','genre','payment','rights','year','publisher','debut','agented','race','gender','sex_orient','disability','auth_name','title','comp_titles','comments']
df.columns = cols
df.head()

,timestamp,genre,payment,rights,year,publisher,debut,agented,race,gender,sex_orient,disability,auth_name,title,comp_titles,comments
0,6/11/2020 7:09:01,Academic,"$1,000",Worldwide,2018,Springer,Yes,No,White,Cis Man,Straight,No,Moorkens Castilho Gaspari Doherty,Translation Quality Assessment,NaN,NaN
1,6/7/2020 0:17:09,Academic,"$3,000",Worldwide,2019,University Press,No,Yes,White,NaN,Straight,No,NaN,NaN,NaN,NaN
2,6/9/2020 7:15:32,Academic,$0,North America Only,2011,Louisiana State University,Yes,No,White,Cis Woman,Straight,No,NaN,NaN,NaN,NaN
3,6/10/2020 9:23:46,Academic,$500,Worldwide,2013,Praeger,No,No,Multiracial,Cis Woman,Straight,No,NaN,NaN,NaN,NaN
4,6/10/2020 9:24:52,Academic,$0,Worldwide,2012,University Press,Yes,No,Multiracial,Cis Woman,Straight,No,NaN,NaN,NaN,NaN


Get list of all non-NA titles

In [ ]:
titles = df.loc[~df.title.isna()]

## Use Goodreads API to get data

**PROBLEM!: Goodreads has depreciated their API, so this is no longer possible**

Going to use this package to get the data:
https://pypi.org/project/Goodreads/

In [ ]:
!pip install Goodreads

In [ ]:
import xmltodict
import requests
import rauth

In [ ]:
from goodreads import client
gc = client.GoodreadsClient('eMGdHafsnpk0r3Qy27yrRg', '0EvpzI1rPanBWkcO23OJcHVkHjldHzxfd0NXAThZI')

In [ ]:
book = gc.book(1)

In [ ]:
book

Harry Potter and the Half-Blood Prince (Harry Potter, #6)

In [ ]:
books = gc.search_books('People Who Knew Me')

In [ ]:
book = books[0]

In [ ]:
book.title

'People Who Knew Me'

In [ ]:
book.authors[0]

Kim Hooper

In [ ]:
book.average_rating

'3.99'

In [ ]:
book.ratings_count

'925'

In [ ]:
book.publisher

"St. Martin's Press"

In [ ]:
book.publication_date

('5', '24', '2016')

In [ ]:
int(book.publication_date[2])

2016

Now let's systematize this a bit

In [ ]:
def pull_top_matches(title, number_matches):
  records = list()
  try:
    books = gc.search_books(title)
    for book in books[:number_matches]:
      rec = dict()
      rec['search_term'] = title
      rec['title'] = book.title
      rec['author'] = book.authors[0]
      rec['publisher'] = book.publisher
      rec['ratings_count'] = book.ratings_count
      rec['average_rating'] = book.average_rating
      rec['pub_date'] = book.publication_date
      rec['pub_year'] = book.publication_date[2]
      records.append(rec)
      # import pdb; pdb.set_trace()
  except:
      print("error: ",title)
  df = pd.DataFrame.from_records(records)
  return df


Is there any difference in time it takes to pull more results? 

In [ ]:
import time

start = time.time()

tmp1 = pull_top_matches('People Who Knew Me', 3)

end = time.time()
print(end-start)

9.722641229629517


In [ ]:
start = time.time()

tmp2 = pull_top_matches('People Who Knew Me', 25)

end = time.time()
print(end-start)

9.676860332489014


Doesn't seem like there is any time difference.

In [ ]:
pull_top_matches('The Wild Inside', 3)

,search_term,title,author,publisher,ratings_count,average_rating,pub_date,pub_year
0,The Wild Inside,The Wild Inside (Glacier Mystery #1),Christine Carbo,Atria Books,2289,3.62,"(6, 16, 2015)",2015
1,The Wild Inside,The Operators: The Wild and Terrifying Inside ...,Michael Hastings,"Little, Brown & Company",1702,3.93,"(12, 6, 2011)",2011
2,The Wild Inside,The Wild Inside,Jamey Bradbury,William Morrow,574,3.29,"(3, 20, 2018)",2018


We can really narrow the search results if we include the first word in the authors column

In [ ]:
search_term = titles.iloc[1].title + ' ' + titles.iloc[1].auth_name
pull_top_matches(search_term, 3)

,search_term,title,author,publisher,ratings_count,average_rating,pub_date,pub_year
0,The Wild Inside Jamey Bradbury,The Wild Inside,Jamey Bradbury,William Morrow,574,3.29,"(3, 20, 2018)",2018


This also seems to speed up the search process a lot

In [ ]:
# Create a variable that combines author and title
titles['auth_title'] = titles.title + ' ' + titles.auth_name

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
titles.loc[titles.auth_title.str.contains('The Wild', na=False)]

,timestamp,genre,payment,rights,year,publisher,debut,agented,race,gender,sex_orient,disability,auth_name,title,comp_titles,comments,payment_cleaned,genre_clean,publisher_clean,race_group,race_group_detail,payment_category,auth_title
6,6/10/2020 15:30:39,Adult Literary Fiction,"$285,000",English Language Only,2018,HarperCollins (and Imprints),Yes,Yes,White,Cis Woman,Demi/Asexual,No,Jamey Bradbury,The Wild Inside,NaN,NaN,285000.000,Literary,Big 5,White,White,$100k+,The Wild Inside Jamey Bradbury
899,6/10/2020 6:59:01,"Non-Fiction, Audio","$473,000",Worldwide,2015,Penguin Random House (and Imprints),No,Yes,White,Cis Man,Straight,No,Abel James,The Wild Diet,NaN,"Multiple offers from bidding, many in the 300-...",473000.000,Non-Fiction,Big 5,White,White,$100k+,The Wild Diet Abel James


Now iterate over all of the books in the list

In [ ]:
import tqdm

book_list = list(titles.auth_title[0:10])
results = list()

# iterate
for auth_title in tqdm.tqdm(book_list): 
  print(auth_title)
  result = pull_top_matches(auth_title, number_matches=3)
  results.append(result)

# concatenate
gr_results = pd.concat(results)



  0%|          | 0/10 [00:00<?, ?it/s]

Translation Quality Assessment Moorkens Castilho Gaspari Doherty


 10%|█         | 1/10 [00:01<00:11,  1.25s/it]

The Wild Inside Jamey Bradbury


 20%|██        | 2/10 [00:02<00:08,  1.11s/it]

...

 80%|████████  | 8/10 [00:07<00:01,  1.09it/s]

Loose Lips Amy Stephenson


 90%|█████████ | 9/10 [00:08<00:00,  1.11it/s]

Moonflower, Nightshade, All the Hours of the Day JD Scott


100%|██████████| 10/10 [00:09<00:00,  1.04it/s]


In [ ]:
vars = ['title','auth_name','publisher','year','payment']
titles[vars].head(5)

,title,auth_name,publisher,year,payment
0,Translation Quality Assessment,Moorkens Castilho Gaspari Doherty,Springer,2018,"$1,000"
6,The Wild Inside,Jamey Bradbury,HarperCollins (and Imprints),2018,"$285,000"
7,People Who Knew Me,Kim Hooper,St. Martin's Press,2016,"$25,000"
8,Cherry Blossoms,Kim Hooper,Turner,2018,"$5,000"
9,Tiny,Kim Hooper,Turner,2019,"$5,000"


In [ ]:
gr_results.head(5)

,search_term,title,author,publisher,ratings_count,average_rating,pub_date,pub_year
0,Translation Quality Assessment Moorkens Castil...,Translation Quality Assessment: From Principle...,Joss Moorkens,Springer,1,5.00,"(9, 12, 2018)",2018
1,Translation Quality Assessment Moorkens Castil...,Translation Quality Assessment: From Principle...,Joss Moorkens,None,0,0.00,"(None, None, None)",None
0,The Wild Inside Jamey Bradbury,The Wild Inside,Jamey Bradbury,William Morrow,574,3.29,"(3, 20, 2018)",2018
0,People Who Knew Me Kim Hooper,People Who Knew Me,Kim Hooper,St. Martin's Press,925,3.99,"(5, 24, 2016)",2016
0,Cherry Blossoms Kim Hooper,Cherry Blossoms,Kim Hooper,Turner,26,4.30,"(10, 30, 2018)",2018


Create a function that will do this in chunks to prevent timeout issues. This is probably no longer necessary given the speedup I've realised with the search function.

In [ ]:
from google.colab import files

In [ ]:
chunks = [list(titles.auth_title[x:x+200]) for x in range(0, len(titles.auth_title), 200)]
chunk_names = [0,1,2,3]

In [ ]:
for name, books_chunk in zip(chunk_names, chunks):

  book_list = books_chunk
  results = list()

  # iterate
  for title in tqdm.tqdm(book_list): 
    print(title)
    result = pull_top_matches(title, number_matches=25)
    results.append(result)

  # concatenate
  gr_results = pd.concat(results)
  gr_results.to_csv('goodreads_data_'+str(name)+'.csv')
  files.download('goodreads_data_'+str(name)+'.csv')

  0%|          | 0/200 [00:00<?, ?it/s]

Translation Quality Assessment Moorkens Castilho Gaspari Doherty


  0%|          | 1/200 [00:01<04:01,  1.21s/it]

The Wild Inside Jamey Bradbury


  1%|          | 2/200 [00:01<03:34,  1.08s/it]

...

 95%|█████████▍| 18/19 [00:17<00:00,  1.02it/s]

Son of the Storm (The Nameless Republic #1) Suyi Davies Okungbowa


100%|██████████| 19/19 [00:18<00:00,  1.01it/s]


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Now that data has loaded access all files from github and join to dataframe

**Project**: Get data to autosave to GitHub:

https://towardsdatascience.com/google-drive-google-colab-github-dont-just-read-do-it-5554d5824228

https://zerowithdot.com/colab-github-workflow/

## Save Data to GitHub

Create a new csv file of random data and then save it to drive and upload to github

In [ ]:
tmp = {'a':[1,2,3,4,5],'b':[1,2,3,4,5]}
tmp = pd.DataFrame(tmp)
tmp.head()

,a,b
0,1,1
1,2,2
2,3,3
3,4,4
4,5,5


In [ ]:
%cd  ./tmp
!ls

/content/tmp
temp_file  test


In [ ]:
tmp.to_csv('./temp_file')

Now we need to save this file into Google Drive

Step 1: copy new files over to Google Drive

In [ ]:
!cp -r ./tmp/* "{PROJECT_PATH}"

In [ ]:
!ls '{PROJECT_PATH}'

 books.csv		     goodreads_data_2.csv	 LICENSE
'goodreads_data_0 (1).csv'  'goodreads_data_3 (1).csv'	 ppm_20201010.csv
 goodreads_data_0.csv	     goodreads_data_3.csv	 temp_file
'goodreads_data_1 (1).csv'   goodreads_data_4.csv	 test
 goodreads_data_1.csv	     goodreads_data_5.csv
'goodreads_data_2 (1).csv'   goodreads_data_6.csv


Step 2: resubmit to GitHub

In [ ]:
GIT_COMMIT_MESSAGE = 'Testing Colab commit to GitHub'
GIT_EMAIL = 'robert@sturrock.ca' 
GIT_NAME = 'Robert Sturrock'
GIT_BRANCH_NAME = 'master'

In [ ]:
!ls

drive  sample_data


In [ ]:
!git clone "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" ./tmp/publishing-paid-me
!rsync -aP --exclude=data/ "{PROJECT_PATH}"/* ./tmp/publishing-paid-me # this gets all the files in drive I think. You'd need a folder "data/" to exclude

%cd ./tmp/publishing-paid-me
!git add .
!git commit -m '"{GIT_COMMIT_MESSAGE}"'
!git config --global user.email "{GIT_EMAIL}"
!git config --global user.name "{GIT_NAME}"
!git push origin "{GIT_BRANCH_NAME}"
%cd /content

# Remove directory
!rm -r ./tmp

Cloning into './tmp/publishing-paid-me'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 9), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (35/35), done.
sending incremental file list
LICENSE
         35,129 100%    2.25MB/s    0:00:00 (xfr#1, to-chk=15/16)
books.csv
      1,559,650 100%  165.27MB/s    0:00:00 (xfr#2, to-chk=14/16)
goodreads_data_0 (1).csv
         51,707 100%    4.93MB/s    0:00:00 (xfr#3, to-chk=13/16)
goodreads_data_0.csv
        170,457 100%   12.50MB/s    0:00:00 (xfr#4, to-chk=12/16)
goodreads_data_1 (1).csv
         65,978 100%    4.49MB/s    0:00:00 (xfr#5, to-chk=11/16)
goodreads_data_1.csv
        126,751 100%    7.55MB/s    0:00:00 (xfr#6, to-chk=10/16)
goodreads_data_2 (1).csv
         46,758 100%    2.62MB/s    0:00:00 (xfr#7, to-chk=9/16)
goodreads_data_2.csv
        118,738 100%    5.96MB/s    0:00:00 (xfr#8, to-chk=8/16)
goodreads

## Bonus: Remove Files from GitHub

I can also remove files from GitHub

1. Single File

In [ ]:
!git clone "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" ./tmp/publishing-paid-me # get git repo
!rsync -aP "{PROJECT_PATH}"/* ./tmp/publishing-paid-me # this gets all the files in drive I think.

# Git hub changes
print("GitHub Section")
%cd ./tmp/publishing-paid-me
!git rm goodreads_data_0.csv
!git commit -m "remove goodreads file (test)"
!git push origin "{GIT_BRANCH_NAME}"
%cd /content

# Remove directory
!rm -r ./tmp

Cloning into './tmp/publishing-paid-me'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 35 (delta 9), reused 11 (delta 2), pack-reused 0
Unpacking objects: 100% (35/35), done.
sending incremental file list
LICENSE
         35,129 100%    2.25MB/s    0:00:00 (xfr#1, to-chk=15/16)
books.csv
      1,559,650 100%  185.92MB/s    0:00:00 (xfr#2, to-chk=14/16)
goodreads_data_0 (1).csv
         51,707 100%    4.93MB/s    0:00:00 (xfr#3, to-chk=13/16)
goodreads_data_0.csv
        170,457 100%   13.55MB/s    0:00:00 (xfr#4, to-chk=12/16)
goodreads_data_1 (1).csv
         65,978 100%    4.84MB/s    0:00:00 (xfr#5, to-chk=11/16)
goodreads_data_1.csv
        126,751 100%    8.06MB/s    0:00:00 (xfr#6, to-chk=10/16)
goodreads_data_2 (1).csv
         46,758 100%    2.79MB/s    0:00:00 (xfr#7, to-chk=9/16)
goodreads_data_2.csv
        118,738 100%    6.66MB/s    0:00:00 (xfr#8, to-chk=8/16)
goodreads

2. Multiple files

In [ ]:
!git clone "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" ./tmp/publishing-paid-me # get git repo
# !rsync -aP "{PROJECT_PATH}"/* ./tmp/publishing-paid-me # this gets all the files in drive I think.

print("GitHub Section")
%cd ./tmp/publishing-paid-me
!git rm goodreads_data_*.csv
!git commit -m "remove goodreads file (test)"
!git push origin "{GIT_BRANCH_NAME}"
%cd /content

# Remove directory
!rm -r ./tmp

Cloning into './tmp/publishing-paid-me'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 37 (delta 10), reused 13 (delta 3), pack-reused 0
Unpacking objects: 100% (37/37), done.
GitHub Section
/content/tmp/publishing-paid-me
rm 'goodreads_data_0 (1).csv'
rm 'goodreads_data_1 (1).csv'
rm 'goodreads_data_1.csv'
rm 'goodreads_data_2 (1).csv'
rm 'goodreads_data_2.csv'
rm 'goodreads_data_3 (1).csv'
rm 'goodreads_data_3.csv'
rm 'goodreads_data_4.csv'
rm 'goodreads_data_5.csv'
rm 'goodreads_data_6.csv'
[master d5272fd] remove goodreads file (test)
 10 files changed, 5971 deletions(-)
 delete mode 100644 goodreads_data_0 (1).csv
 delete mode 100644 goodreads_data_1 (1).csv
 delete mode 100644 goodreads_data_1.csv
 delete mode 100644 goodreads_data_2 (1).csv
 delete mode 100644 goodreads_data_2.csv
 delete mode 100644 goodreads_data_3 (1).csv
 delete mode 100644 goodreads_data_3.csv
 delete mod

## Bonus: Git revert 

Have to  do this since unfortunately the Goodreads API is now dead.

In [ ]:
GIT_COMMIT_MESSAGE = "rolling back delete of data as Goodreads API is broken"
GIT_EMAIL = 'robert@sturrock.ca' 
GIT_NAME = 'Robert Sturrock'
GIT_BRANCH_NAME = 'master'

In [ ]:
GIT_USERNAME = "robert-sturrock" 
GIT_TOKEN = "50391de799dba17eec450691a028bc5319e4412a" 
GIT_REPOSITORY = "publishing-paid-me"    

# get repository
!git clone "https://{GIT_TOKEN}@github.com/{GIT_USERNAME}/{GIT_REPOSITORY}.git" ./tmp/publishing-paid-me # get git repo

# revert
%cd ./tmp/publishing-paid-me
!git revert '998cd0dee604fbd1deecfe513ac52d901b535f60' # commit hash
!git commit -m '"{GIT_COMMIT_MESSAGE}"'
!git config --global user.email "{GIT_EMAIL}"
!git config --global user.name "{GIT_NAME}"
!git push origin "{GIT_BRANCH_NAME}"


Cloning into './tmp/publishing-paid-me'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 42 (delta 13), reused 25 (delta 5), pack-reused 0
Unpacking objects: 100% (42/42), done.
/content/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me/tmp/publishing-paid-me
hint: Waiting for your editor to close the file... error: unable to start editor 'editor'
Please supply the message using either -m or -F option.
[master c932905] "rolling back delete of data as Goodreads API is broken"
 1 file changed, 1220 insertions(+)
 create mode 100644 goodreads_data_0.csv
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 56.45 KiB | 5.13 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
rem